# FILTER

In [ ]:
impor pandas as pd

In [ ]:
input_merged_summary=snakemake.input.merged_summary
input_vibrant_circular=snakemake.input.vibrant_circular
input_vibrant_positive=snakemake.input.vibrant_positive
input_vibrant_quality=snakemake.input.vibrant_quality
input_vibrant_summary=snakemake.input.vibrant_summary
input_virsorter_table=snakemake.input.virsorter_table
input_virsorter_positive_list=snakemake.input.virsorter_positive_list
input_genomad_virus_summary=snakemake.input.genomad_virus_summary
input_genomad_plasmid_summary=snakemake.input.genomad_plasmid_summary
input_genomad_viral_fasta=snakemake.input.genomad_viral_fasta
input_genomad_viral_fasta_conservative=snakemake.input.genomad_viral_fasta_conservative
#________
output_test=snakemake.output.fake

open(output_test, mode='a').close()

In [ ]:

checkv_df_vOTUs = pd.read_csv(input_merged_summary, sep="\t",
                            names= ["contig_id_vOTU", "contig_length_vOTU", "provirus_vOTU", "proviral_length_vOTU",
                           "gene_count_vOTU", "viral_genes_vOTU", "host_genes_vOTU", "checkv_quality_vOTU",
                           "miuvig_quality_vOTU", "completeness_vOTU", "completeness_method_vOTU",
                           "contamination_vOTU", "kmer_freq_vOTU", "warnings_vOTU"])
checkv_df_vOTUs